In [1]:
import pandas as pd

In [2]:
# read in data
movie_start_df = pd.read_csv("output/clean_df.csv")
movie_list_df = pd.read_csv("output/movie_table.csv")

movie_start_df.head(5)

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating,boxoffice
0,0,Pirates of the Caribbean: Dead Man's Chest,PG-13,2006-07-07,151,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Stuart Beattie","Johnny Depp, Orlando Bloom, Keira Knightley","English, Turkish, Greek, Mandarin, French",https://m.media-amazon.com/images/M/MV5BMTcwOD...,53.0,7.3,423315812
1,1,Night at the Museum,PG,2006-12-22,108,"Adventure, Comedy, Family",Shawn Levy,"Robert Ben Garant, Thomas Lennon, Milan Trenc","Ben Stiller, Carla Gugino, Ricky Gervais","English, Italian, Hebrew",https://m.media-amazon.com/images/M/MV5BMTQyOT...,48.0,6.4,250863268
2,2,Cars,G,2006-06-09,117,"Animation, Adventure, Comedy","John Lasseter, Joe Ranft","John Lasseter, Joe Ranft, Jorgen Klubien","Owen Wilson, Bonnie Hunt, Paul Newman","English, Italian, Japanese, Yiddish",https://m.media-amazon.com/images/M/MV5BMTg5Nz...,73.0,7.1,244082982
3,3,X-Men: The Last Stand,PG-13,2006-05-26,104,"Action, Adventure, Sci-Fi",Brett Ratner,"Simon Kinberg, Zak Penn","Patrick Stewart, Hugh Jackman, Halle Berry",English,https://m.media-amazon.com/images/M/MV5BNDBhND...,58.0,6.7,234362462
4,4,The Da Vinci Code,PG-13,2006-05-19,149,"Mystery, Thriller",Ron Howard,"Akiva Goldsman, Dan Brown","Tom Hanks, Audrey Tautou, Jean Reno","English, French, Latin, Spanish",https://m.media-amazon.com/images/M/MV5BMjIxMj...,46.0,6.6,217536138


In [3]:
movie_df = pd.merge(movie_start_df, movie_list_df, on=["movie_id"], how="inner")
movie_df.head()

,movie_id,title_x,movie_rating_x,release_date_x,runtime_x,genre_name,Director,Writer,Actors,Language,...,boxoffice_x,title_y,movie_rating_y,release_date_y,runtime_y,metascore_y,imdb_rating_y,budget,boxoffice_y,buzz
0,1,Night at the Museum,PG,2006-12-22,108,"Adventure, Comedy, Family",Shawn Levy,"Robert Ben Garant, Thomas Lennon, Milan Trenc","Ben Stiller, Carla Gugino, Ricky Gervais","English, Italian, Hebrew",...,250863268,Night at the Museum,PG,2006-12-22,108.0,48.0,6.4,110000000,250863268,6
1,2,Cars,G,2006-06-09,117,"Animation, Adventure, Comedy","John Lasseter, Joe Ranft","John Lasseter, Joe Ranft, Jorgen Klubien","Owen Wilson, Bonnie Hunt, Paul Newman","English, Italian, Japanese, Yiddish",...,244082982,Cars,G,2006-06-09,117.0,73.0,7.1,70000000,244082982,4
2,3,X-Men: The Last Stand,PG-13,2006-05-26,104,"Action, Adventure, Sci-Fi",Brett Ratner,"Simon Kinberg, Zak Penn","Patrick Stewart, Hugh Jackman, Halle Berry",English,...,234362462,X-Men: The Last Stand,PG-13,2006-05-26,104.0,58.0,6.7,210000000,234362462,23
3,4,The Da Vinci Code,PG-13,2006-05-19,149,"Mystery, Thriller",Ron Howard,"Akiva Goldsman, Dan Brown","Tom Hanks, Audrey Tautou, Jean Reno","English, French, Latin, Spanish",...,217536138,The Da Vinci Code,PG-13,2006-05-19,149.0,46.0,6.6,125000000,217536138,7
4,5,Superman Returns,PG-13,2006-06-28,154,"Action, Sci-Fi",Bryan Singer,"Michael Dougherty, Dan Harris, Bryan Singer","Brandon Routh, Kevin Spacey, Kate Bosworth","English, German, French",...,200081192,Superman Returns,PG-13,2006-06-28,154.0,72.0,6.0,232000000,200120000,15


In [4]:
# create dfs for tables
role_df = pd.DataFrame({"role_id": [0,1,2], "role": ["Director", "Writer", "Actor"]})
person_df = pd.DataFrame({"person_id": [], "person": []})
movie_role_person_df = pd.DataFrame(columns=["movie", "person", "role"])

In [5]:
# set indexes
role_df.set_index(['role_id'], inplace=True)
person_df.set_index(["person_id"], inplace=True)
role_df

,role
role_id,
0,Director
1,Writer
2,Actor


In [6]:
person_df.head()

,person
person_id,


In [7]:
# iterate through movie_df
for index, row in movie_df.iterrows():
    # split out all people with roles
    directors = row["Director"].split(", ")
    writers = row["Writer"].split(", ")
    actors = row["Actors"].split(", ")


    for director in directors:
        # check if person in person_df
        if director not in person_df["person"].values:
            new_person_row = {"person": director}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 0, "person": person_df.index[person_df["person"] == director].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)
        
    for writer in writers:
        # check if person in person_df
        if writer not in person_df["person"].values:
            new_person_row = {"person": writer}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 1, "person": person_df.index[person_df["person"] == writer].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)

    for actor in actors:
        # check if person in person_df
        if actor not in person_df["person"].values:
            new_person_row = {"person": actor}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 2, "person": person_df.index[person_df["person"] == actor].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)

C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # This is added back by InteractiveShellApp.init_path()


In [8]:
person2 = person_df.rename(columns={'person':'name'})
person2.index.name = "person_id"
person2.head()

,name
person_id,
0,Shawn Levy
1,Robert Ben Garant
2,Thomas Lennon
3,Milan Trenc
4,Ben Stiller


In [9]:
# converts person column from list type to string type
movie_role_person_df['person']  = [','.join(map(str, row)) for row in movie_role_person_df['person']]

In [10]:
movie_role_person_df2 = movie_role_person_df.rename(columns={'movie':'movie_id', 'person':'person_id', 'role':'role_id'})
movie_role_person_df2

,movie_id,person_id,role_id
0,1,0,0
1,1,1,1
2,1,2,1
3,1,3,1
4,1,4,2
...,...,...,...
10118,6854,6065,1
10119,6854,6066,1
10120,6854,6067,2
10121,6854,6068,2


In [11]:
role_df.to_csv("output/role_table.csv")
person2.to_csv("output/person_table.csv")
movie_role_person_df2.to_csv("output/movie_role_person_junction_table.csv", index=False)